In [1]:
#!pip install --upgrade pip

In [2]:
!pip install pandas pyarrow scipy

In [3]:
# function to download data.

import requests
from tqdm import tqdm

def get_filename(ticker, interval, year):
    return f'data/{ticker}_{interval}_{year}.parquet'

def download(ticker, interval, year):
    filename = get_filename(ticker, interval, year)
    if year:
      # url for seconds
      url = f"https://barsv.com/api/download/file/{ticker}/{year}?format=parquet&interval={interval}"
    else:
      # url for minutes+
      url = f"https://barsv.com/api/download/file/{ticker}/{ticker}_{interval}?interval={interval}&format=parquet"
    response = requests.get(url, stream=True)

    # Get the total file size from the content-length header
    total_size = int(response.headers.get('content-length', 0))

    with open(filename, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            bar.update(size)

In [4]:
ticker = 'AAPL'
interval = '5s'
year = '2024'

In [5]:
import os

filename = get_filename(ticker, interval, year)
# If file doesn't exist, then download it.
if not os.path.exists(filename):
	download(ticker, interval, year)

In [6]:
# plot last points.
import pandas as pd
import plotly.express as px

filename = get_filename(ticker, interval, year)
df = pd.read_parquet(filename)

fig = px.line(df[-1000:], y='open', title=f'{ticker} Open Prices')
fig.show()

In [7]:
total_bars = df.shape[0]
print(f'Total bars: {total_bars} ({total_bars:,})')

Total bars: 1428355 (1,428,355)


In [8]:
m = 60 # window size

In [9]:
import numpy as np

def create_window(df, index, window_size):
    # note: astype(np.float64) is needed here because later we
    # do `m * sum_a_sq - sum_a**2` that in float32 is always zero
    # due to floating-point inaccuracy.
    return df.loc[index:index + window_size - 1]['open'].astype(np.float64).values


In [10]:
a0 = create_window(df, 42, m)
a = create_window(df, 82997, m)

In [11]:
# plot the original data without re-scalling.

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"rowspan": 2}, {"rowspan": 1}],
           [None, {}]],
    subplot_titles=("Shared Y-axis", "a0 (Separate Y-axis)", "a (Separate Y-axis)")
)

# Plot on the left (shared y-axis)
fig.add_trace(go.Scatter(y=a0, name='a0'), row=1, col=1)
fig.add_trace(go.Scatter(y=a, name='a'), row=1, col=1)

# Plot on the right (separate y-axes)
fig.add_trace(go.Scatter(y=a0, name='a0'), row=1, col=2)
fig.add_trace(go.Scatter(y=a, name='a'), row=2, col=2)

fig.update_layout(margin=dict(b=100))  # Увеличь отступ снизу

fig.add_annotation(
    text="Fig. 1. Here a and a0 are plotted on the same Y scale.",
    xref="paper", yref="paper",
    x=0, y=-0.20,
    showarrow=False,
    font=dict(size=16),
)

fig.add_annotation(
    text="Fig. 2. Here a and a0 are plotted each on it's own Y scale.",
    xref="paper", yref="paper",
    x=0.8, y=-0.20,
    showarrow=False,
    font=dict(size=16),
)
fig.show()

In [12]:
# implementation of the above formulas.
def get_alpha_lambda(a0, a, sum_a0 = None):
    ''' sum_a0 is to speed up calculations when we have 1 a0 and many a.'''
    m = len(a)
    sum_a = a.sum()
    if sum_a0 is None:
        sum_a0 = a0.sum()
    sum_a_sq = (a**2).sum()
    sum_a_a0 = (a * a0).sum()

    divider = m * sum_a_sq - sum_a**2
    if divider == 0:
        #print(f'divider: {divider}, m: {m}, sum_a: {sum_a}, sum_a_sq: {sum_a_sq}, a: {a}')
        return np.nan, np.nan

    alpha = (m * sum_a_a0 - sum_a * sum_a0) / divider
    lmbda = (sum_a0 - alpha * sum_a) / m
    return alpha, lmbda

In [13]:
# now rescale.

import numpy as np

def plot_rescaled(a0, a):
    alpha, lmbda = get_alpha_lambda(a0, a)
    print(f'alpha = {alpha}')
    print(f'lambda = {lmbda}')

    # Apply the transformation
    a_transformed = alpha * a + lmbda

    # Plot the original and transformed data
    import plotly.graph_objects as go

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=a0, name='a0'))
    fig.add_trace(go.Scatter(y=a_transformed, name='a (transformed)'))
    fig.add_annotation(
        text="Fig. 3. Here a is rescaled to match a0. NOTE: negative alpha flips a.",
        xref="paper", yref="paper",
        x=0.5, y=-0.20,
        showarrow=False,
        font=dict(size=16),
    )
    fig.show()

plot_rescaled(a0, a)


alpha = 2.215449329453228
lambda = -241.14381463521707


In [14]:
import numpy as np

def get_rmse(a0, a):
  """Calculates the root mean square error of the linear fit between two series.

  Args:
    a0: The first pandas Series.
    a: The second pandas Series.

  Returns:
    The root mean square error.
  """
  alpha, lmbda = get_alpha_lambda(a0, a)
  if np.isnan(alpha):
      return 1e9 # a large number
  a_transformed = alpha * a + lmbda
  return np.sqrt(np.mean((a0 - a_transformed)**2))

print(f'get_rmse(a0, a): {get_rmse(a0, a)}')

get_rmse(a0, a): 0.032197960761568785


In [15]:
from scipy import signal

class PatternSearcher:
    """
    A class to pre-calculate cumulative sums of a time series for optimized
    pattern searching using Normalized Cross-Correlation.
    """
    def __init__(self, series):
        """
        Initializes the PatternSearcher with a time series.

        Args:
            series: The time series data (pandas Series or NumPy array).
        """
        # Convert the series to a NumPy array with float64 dtype for numerical stability
        self.series = np.asarray(series, dtype=np.float64)
        self.n = len(self.series)

        # Pre-calculate cumulative sums
        self.cum_sum_x = np.cumsum(self.series)
        self.cum_sum_x_sq = np.cumsum(self.series**2)

    def search(self, template):
        """
        Finds the best match of a template pattern within the time series
        using Normalized Cross-Correlation.

        Args:
            template: The pattern template to search for (pandas Series or NumPy array).

        Returns:
            A tuple containing the index of the best match and its correlation
            coefficient, or None if the template has zero variance.
        """
        # Convert the template to a NumPy array with float64 dtype
        template = np.asarray(template, dtype=np.float64)
        m = len(template)

        if m > self.n:
            print("Template length is greater than series length.")
            return None

        # --- 2. Calculate terms for the template (y) ---
        sum_y = template.sum()
        sum_y_sq = (template**2).sum()
        # Denominator term for y, which is constant
        denom_y = m * sum_y_sq - sum_y**2

        if denom_y <= 1e-10: # Use a small epsilon for floating point comparison
            print("Template has zero variance, cannot perform matching.")
            return None

        # --- 3. Calculate terms for the series (x) using rolling sums ---
        # Rolling sum of x for each window
        sum_x = self.cum_sum_x[m-1:] - np.concatenate(([0], self.cum_sum_x[:-m]))
        # Rolling sum of x^2 for each window
        sum_x_sq = self.cum_sum_x_sq[m-1:] - np.concatenate(([0], self.cum_sum_x_sq[:-m]))

        # Denominator term for x for each window
        denom_x = m * sum_x_sq - sum_x**2
        # Avoid division by zero for windows with no variance
        denom_x[denom_x <= 1e-10] = 1 # Set to 1 to avoid NaN, result will be 0 anyway

        # --- 4. Calculate cross-correlation term sum(xy) ---
        # This is the sum(x*y) term, calculated efficiently using FFT
        sum_xy = signal.fftconvolve(self.series, template[::-1], mode='valid')

        # --- 5. Calculate Correlation Coefficient (r) for all windows ---
        # Numerator term
        num = m * sum_xy - sum_x * sum_y
        # Denominator term
        denom = np.sqrt(denom_x * denom_y)

        # Handle potential division by zero in denom
        r = np.zeros_like(num, dtype=np.float64) # Initialize r with zeros
        non_zero_denom_mask = denom > 1e-10 # Create a mask for non-zero denominators
        # Calculate r only where denom is not zero
        r[non_zero_denom_mask] = num[non_zero_denom_mask] / denom[non_zero_denom_mask]

        return r

    def get_sorted_r(self, template, return_top_k=None):
        """
        Get sorted correlation results with optional optimization for top-K results.
        
        Args:
            template: The pattern template to search for.
            return_top_k: If specified, return only the top K results using argpartition
                         for better performance. If None, return all results sorted.
        
        Returns:
            List of tuples (index, correlation_value) sorted by absolute correlation.
        """
        r = self.search(template)
        if r is None:
            return None
            
        # OPTIMIZATION: Use argpartition for top-K selection instead of full sorting
        if return_top_k is not None and return_top_k < len(r):
            # Use argpartition to find top-K efficiently (O(n) vs O(n log n))
            abs_r = np.abs(r)
            top_k_indices = np.argpartition(abs_r, -return_top_k)[-return_top_k:]
            # Sort only the top-K results
            sorted_within_top = np.argsort(abs_r[top_k_indices])[::-1]
            final_indices = top_k_indices[sorted_within_top]
            top_r_values = r[final_indices]
            return list(zip(final_indices, top_r_values))
        else:
            # Full sorting (original behavior)
            top_indices = np.argsort(np.abs(r))[::-1]
            top_r_values = r[top_indices]
            return list(zip(top_indices, top_r_values))

## Run the experiment

### Subtask:
Run the experiment: Instantiate this searcher, loop through 100 randomly selected patterns from the dataset, and use the searcher to find the best match for each one.


In [16]:
from scipy import signal
import random
from tqdm import tqdm

# Instantiate the optimized PatternSearcher with the 'open' column of the DataFrame
searcher = PatternSearcher(df['open'])
# Set the random seed for reproducibility
random.seed(42)
correlations = []

In [17]:
N = 100
# get correlations for N randomly sampled patterns
for _ in tqdm(range(N)):
    start_index = random.randrange(0, len(df) - m)
    pattern = create_window(df, start_index, m)
    correlations.append({
        'start_index': start_index,
        'top_10k_r': searcher.get_sorted_r(pattern, 10000)
    })


100%|██████████| 100/100 [00:11<00:00,  8.56it/s]


In [18]:
# take slice_point-th correlation in each experiment, sort them, plot
slice_point = 9900
correlations.sort(key=lambda x: abs(x['top_10k_r'][slice_point][1]), reverse=True)

y = []
for corr in correlations:
    y.append(abs(corr['top_10k_r'][slice_point][1]))

fig = go.Figure()
fig.add_trace(go.Scatter(y=y, name=f'slice_point-th r values sorted'))
fig.show()

In [19]:
import plotly.express as px

# Extract the 1000th correlation value from each experiment
y = [abs(corr['top_10k_r'][slice_point][1]) for corr in correlations]

# Create the histogram
fig = px.histogram(x=y, nbins=40, title='Distribution of the 1000th Correlation Value')
fig.update_layout(
    xaxis_title="Absolute Correlation Value",
    yaxis_title="Frequency",
)
fig.show()

In [20]:
for c in correlations[:2]:
  a = create_window(df, c['start_index'], m)
  fig = go.Figure()
  fig.add_trace(go.Scatter(y=a, name='a'))
  fig.show()

In [21]:
for c in correlations[1]['top_10k_r'][:10]:
  print(c)
  a = create_window(df, c[0], m)
  fig = go.Figure()
  fig.add_trace(go.Scatter(y=a, name='a'))
  fig.show()

(np.int64(477937), np.float64(0.9999993357830606))


(np.int64(477938), np.float64(0.9914787125195209))


(np.int64(477936), np.float64(0.9904447378524049))


(np.int64(1244155), np.float64(-0.9881041756107449))


(np.int64(1368072), np.float64(-0.9878302760649145))


(np.int64(644043), np.float64(0.9875231895212967))


(np.int64(456236), np.float64(-0.9873773084652213))


(np.int64(1303792), np.float64(0.987316175258418))


(np.int64(1192109), np.float64(0.9872938798913925))


(np.int64(1244154), np.float64(-0.9870977279070232))
